In [1]:
# Лабораторная работа № 2. Вариант 6 (классы 1,15,16: alt.atheism, sci.space, soc.religion.christian)

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


In [2]:
# Выбор классов
categories = ['alt.atheism', 'sci.space', 'soc.religion.christian']
remove = ('headers', 'footers', 'quotes')
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=24, categories=categories, remove=remove)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=24, categories=categories, remove=remove)
print(twenty_train.data[3])
print(twenty_test.data[3])

If gamma ray bursters are extragalactic, would absorption from the
galaxy be expected?  How transparent is the galactic core to gamma
rays?

How much energy does a burster put out?  I know energy depends on
distance, which is unknown.  An answer of the form _X_ ergs per
megaparsec^2 is OK.




Yes a flotation tank, combined with floride breathing water(REF: the Abyss
breathing solution I think).. also the right position of the astronaut and
strapping you can probably get much more than 45gs in an accesloration..
More like near 100g (or somewhat less)..

Saw I book called the "Time Master" (I thjink that was the title) that had some
ideas on how fast and all you could go..


In [3]:
# Применение стемминга
def stemn(data):
    porter_stemmer = PorterStemmer()
    stem = []
    for text in data:
        nltk_tokens = word_tokenize(text)
        line = ''.join([' ' + porter_stemmer.stem(word) for word in nltk_tokens])
        stem.append(line)
    return stem

porter_stemmer = PorterStemmer()
stem_train = []

In [4]:
for text in twenty_train.data:
    nltk_tokens = word_tokenize(text)
    line = ''
    for word in nltk_tokens:
        line += ' ' + porter_stemmer.stem(word)
    stem_train.append(line)
print(stem_train[0])
stem_test = stemn(twenty_test.data)

 ann jackson ( ajackson @ cs.ubc.ca ) wrote on 5 may : i would like to submit the follow which help me enorm . if it ha alreadi been post , i apolog . it seem that dure the middl age , it wa customari for pastor to explain the triniti to their parishon by analog to water . water is water , but can exist in three form -- liquid , ice and vapor . thu it is possibl for one essenc to exist in three form . and recent , the pastor of my church drew an analog , which i also found use -- a woman is often perciev by other in three way , depend on their relationship to her -- a mother , a wife and an employe in a busi . thu , it seem clear to me that the essenc of god can subsist in the father , son , and holi spirit or , depend on one 's particular need for him .


In [32]:
# Векторизация 
vect = CountVectorizer(max_features=10000, stop_words='english')
vect.fit(twenty_train.data)
train_data = vect.transform(twenty_train.data)
test_data = vect.transform(twenty_test.data)

x = list(zip(vect.get_feature_names_out(), np.ravel(train_data.sum(axis=0))))

def SortbyTF(inputStr):
    return inputStr[1]

x.sort(key=SortbyTF, reverse=True)
print(x[:10])

def tf(input_str):
    return input_str[1]

def terms(vector, data, count):
    x = list(zip(vector.get_feature_names_out(), np.ravel(data.sum(axis=0))))
    x.sort(key=tf, reverse=True)
    return x[:count]

vect_stop = CountVectorizer(max_features=10000, stop_words='english')

train_data_stop = vect_stop.fit_transform(twenty_train.data)
test_data_stop = vect_stop.transform(twenty_test.data)

top_terms_stop = [{term[0]: term[1]} for term in terms(vect_stop, train_data_stop, 20)]
top_terms_stop2 = [{term[0]: term[1]} for term in terms(vect_stop, test_data_stop, 20)]


[('god', 1509), ('space', 1003), ('people', 924), ('don', 699), ('think', 685), ('just', 677), ('like', 649), ('jesus', 621), ('does', 613), ('time', 598)]


In [33]:

# Повторение предыдущего пункта
vect_nostop = CountVectorizer(max_features=10000)

train_data_nostop_stem = vect_nostop.fit_transform(stem_train)
test_data_nostop_stem = vect_nostop.transform(stem_test)

top_terms_stem = [{term[0]: term[1]} for term in terms(vect_nostop, train_data_nostop_stem, 20)]
top_terms_stem_test = [{term[0]: term[1]} for term in terms(vect_nostop, test_data_nostop_stem, 20)]

# Векторизация через TfidgTransformer c использованием (TF и TF-IDF взвешиваний)
tfidf = TfidfTransformer(use_idf=True).fit(train_data)
train_data_tfidf = tfidf.transform(train_data)

# С использованием стоп-слов
train_data_tf = vect_stop.fit_transform(twenty_train.data)
test_data_tf = vect_stop.transform(twenty_test.data)
test_data_tf

train_data_tfidf = tfidf.fit_transform(train_data)
test_data_tfidf = tfidf.transform(test_data)
test_data_tfidf

top_terms_tf = [{term[0]: term[1]} for term in terms(vect_stop, train_data_tf, 20)]
top_terms_tf_test = [{term[0]: term[1]} for term in terms(vect_stop, test_data_tf, 20)]
top_terms_tf_test

top_terms_tfidf = [{term[0]: term[1]} for term in terms(vect_stop, train_data_tfidf, 20)]
top_terms_tfidf_test = [{term[0]: term[1]} for term in terms(vect_stop, test_data_tfidf, 20)]
top_terms_tfidf_test

# Без стоп-слов
train_data_tf = vect_nostop.fit_transform(twenty_train.data)
test_data_tf = vect_nostop.transform(twenty_test.data)

train_data_notfidf = tfidf.fit_transform(train_data)
test_data_notfidf = tfidf.transform(test_data)
test_data_notfidf

top_terms_notf = [{term[0]: term[1]} for term in terms(vect_nostop, train_data_tf, 20)]
top_terms_notf_test = [{term[0]: term[1]} for term in terms(vect_nostop, test_data_tf, 20)]
top_terms_notf_test

top_terms_notfidf = [{term[0]: term[1]} for term in terms(vect_nostop, train_data_tfidf, 20)]
top_terms_notfidf_test = [{term[0]: term[1]} for term in terms(vect_nostop, test_data_tfidf, 20)]
top_terms_notfidf_test

[{'games': 29.90811328943184},
 {'disregard': 20.705849582150545},
 {'jean': 18.540420908716005},
 {'thus': 18.40124460896598},
 {'perceptions': 18.183976195500986},
 {'lasers': 17.800333147859412},
 {'joyfully': 16.690108932221474},
 {'diseases': 14.88421075732181},
 {'sphere': 14.725146919122702},
 {'beams': 14.66841587901161},
 {'tolerate': 14.058060720799915},
 {'charles': 13.633158812340842},
 {'week': 13.183537910602537},
 {'characterize': 13.031761008272525},
 {'designs': 12.685464575051753},
 {'gay': 12.593359632738883},
 {'sma': 12.316941351284537},
 {'seanna': 12.231232969290474},
 {'iraqi': 11.647572890307613},
 {'warning': 11.402873875877027}]

In [34]:
# Вывод таблиц
columns = pd.MultiIndex.from_product([['Count', 'TF', 'TF-IDF'], ['Без стоп-слов', 'С стоп-словами']])
d1 = pd.DataFrame(columns=columns)

d1['Count', 'Без стоп-слов'] = top_terms_stop
d1['TF', 'Без стоп-слов'] = top_terms_notf
d1['TF-IDF', 'Без стоп-слов'] = top_terms_tfidf

d1['Count', 'С стоп-словами'] = top_terms_stop2
d1['TF', 'С стоп-словами'] = top_terms_tf
d1['TF-IDF', 'С стоп-словами'] = top_terms_tfidf

d1


Count                                  TF                    \
       Без стоп-слов      С стоп-словами   Без стоп-слов    С стоп-словами   
0      {'god': 1509}       {'god': 1023}  {'the': 19591}     {'god': 1509}   
1    {'space': 1003}     {'people': 544}   {'of': 10398}   {'space': 1003}   
2    {'people': 924}        {'don': 487}   {'to': 10076}   {'people': 924}   
3       {'don': 699}       {'like': 461}   {'and': 7708}      {'don': 699}   
4     {'think': 685}       {'just': 444}    {'is': 6873}    {'think': 685}   
5      {'just': 677}      {'think': 443}  {'that': 6452}     {'just': 677}   
6      {'like': 649}       {'know': 416}    {'in': 5981}     {'like': 649}   
7     {'jesus': 621}       {'does': 385}    {'it': 4507}    {'jesus': 621}   
8      {'does': 613}       {'time': 376}   {'for': 3357}     {'does': 613}   
9      {'time': 598}        {'say': 341}   {'you': 3343}     {'time': 598}   
10     {'know': 592}    {'believe': 329}   {'not': 3069}     {'know': 592}   
11      {'say': 504}     {'church': 323}    {'be': 2984}      {'say': 504}   
12  {'believe': 496}      {'space': 321}  {'this': 2890}  {'believe': 496}   
13     {'good': 423}        {'way': 321}   {'are': 2777}     {'good': 423}   
14      {'way': 404}      {'jesus': 294}    {'as': 2582}      {'way': 404}   
15     {'nasa': 379}     {'christ': 283}    {'on': 2366}     {'nasa': 379}   
16    {'bible': 371}        {'did': 280}  {'have': 2352}    {'bible': 371}   
17   {'things': 368}       {'good': 264}   {'but': 2047}   {'things': 368}   
18   {'church': 362}        {'new': 246}   {'was': 1960}   {'church': 362}   
19      {'new': 357}  {'christian': 244}  {'with': 1903}      {'new': 357}   

                            TF-IDF                                  
                     Без стоп-слов                  С стоп-словами  
0       {'god': 47.49493085644385}      {'god': 47.49493085644385}  
1   {'people': 32.056927722331345}  {'people': 32.056927722331345}  
2    {'think': 29.217292991820262}   {'think': 29.217292991820262}  
3     {'just': 28.844933151084113}    {'just': 28.844933151084113}  
4      {'don': 28.777536401356738}     {'don': 28.777536401356738}  
5    {'space': 28.404860402000217}   {'space': 28.404860402000217}  
6     {'like': 26.279455417063822}    {'like': 26.279455417063822}  
7       {'know': 25.2494597828208}      {'know': 25.2494597828208}  
8     {'does': 23.563027920793466}    {'does': 23.563027920793466}  
9    {'jesus': 22.530691853323308}   {'jesus': 22.530691853323308}  
10      {'say': 21.01291514223008}      {'say': 21.01291514223008}  
11  {'believe': 20.67859814888153}  {'believe': 20.67859814888153}  
12    {'time': 20.617870223919386}    {'time': 20.617870223919386}  
13    {'bible': 17.91556141148969}    {'bible': 17.91556141148969}  
14    {'good': 17.841453750784684}    {'good': 17.841453750784684}  
15  {'things': 16.869468530029685}  {'things': 16.869468530029685}  
16  {'church': 16.207285629300145}  {'church': 16.207285629300145}  
17     {'way': 16.158158902541853}     {'way': 16.158158902541853}  
18    {'said': 15.694847716790656}    {'said': 15.694847716790656}  
19     {'did': 15.561100251782328}     {'did': 15.561100251782328}

In [35]:
d2 = pd.DataFrame(columns=columns)

d2['Count', 'Без стоп-слов'] = top_terms_stem
d2['TF', 'Без стоп-слов'] = top_terms_tf_test
d2['TF-IDF', 'Без стоп-слов'] = top_terms_tfidf_test

d2['Count', 'С стоп-словами'] = top_terms_stop
d2['TF', 'С стоп-словами'] = top_terms_tf_test
d2['TF-IDF', 'С стоп-словами'] = top_terms_notfidf_test

d2

Count                                    TF                      \
     Без стоп-слов    С стоп-словами       Без стоп-слов      С стоп-словами   
0   {'the': 19591}     {'god': 1509}       {'god': 1023}       {'god': 1023}   
1    {'of': 10398}   {'space': 1003}     {'people': 544}     {'people': 544}   
2    {'to': 10079}   {'people': 924}        {'don': 487}        {'don': 487}   
3    {'and': 7708}      {'don': 699}       {'like': 461}       {'like': 461}   
4     {'is': 7025}    {'think': 685}       {'just': 444}       {'just': 444}   
5   {'that': 6452}     {'just': 677}      {'think': 443}      {'think': 443}   
6     {'in': 5981}     {'like': 649}       {'know': 416}       {'know': 416}   
7     {'it': 4876}    {'jesus': 621}       {'does': 385}       {'does': 385}   
8     {'be': 3511}     {'does': 613}       {'time': 376}       {'time': 376}   
9    {'for': 3357}     {'time': 598}        {'say': 341}        {'say': 341}   
10   {'you': 3343}     {'know': 592}    {'believe': 329}    {'believe': 329}   
11   {'not': 3250}      {'say': 504}     {'church': 323}     {'church': 323}   
12   {'thi': 2895}  {'believe': 496}      {'space': 321}      {'space': 321}   
13   {'are': 2841}     {'good': 423}        {'way': 321}        {'way': 321}   
14    {'as': 2582}      {'way': 404}      {'jesus': 294}      {'jesus': 294}   
15  {'have': 2527}     {'nasa': 379}     {'christ': 283}     {'christ': 283}   
16    {'on': 2372}    {'bible': 371}        {'did': 280}        {'did': 280}   
17   {'but': 2047}   {'things': 368}       {'good': 264}       {'good': 264}   
18    {'do': 2044}   {'church': 362}        {'new': 246}        {'new': 246}   
19    {'wa': 2027}      {'new': 357}  {'christian': 244}  {'christian': 244}   

                               TF-IDF                                        
                        Без стоп-слов                        С стоп-словами  
0          {'god': 29.90811328943184}          {'games': 29.90811328943184}  
1         {'don': 20.705849582150545}     {'disregard': 20.705849582150545}  
2        {'just': 18.540420908716005}          {'jean': 18.540420908716005}  
3        {'think': 18.40124460896598}           {'thus': 18.40124460896598}  
4      {'people': 18.183976195500986}   {'perceptions': 18.183976195500986}  
5        {'like': 17.800333147859412}        {'lasers': 17.800333147859412}  
6        {'know': 16.690108932221474}      {'joyfully': 16.690108932221474}  
7         {'does': 14.88421075732181}       {'diseases': 14.88421075732181}  
8       {'space': 14.725146919122702}        {'sphere': 14.725146919122702}  
9      {'believe': 14.66841587901161}          {'beams': 14.66841587901161}  
10       {'time': 14.058060720799915}      {'tolerate': 14.058060720799915}  
11     {'church': 13.633158812340842}       {'charles': 13.633158812340842}  
12        {'way': 13.183537910602537}          {'week': 13.183537910602537}  
13  {'christian': 13.031761008272525}  {'characterize': 13.031761008272525}  
14        {'did': 12.685464575051753}       {'designs': 12.685464575051753}  
15       {'good': 12.593359632738883}           {'gay': 12.593359632738883}  
16        {'sin': 12.316941351284537}           {'sma': 12.316941351284537}  
17        {'say': 12.231232969290474}        {'seanna': 12.231232969290474}  
18      {'jesus': 11.647572890307613}         {'iraqi': 11.647572890307613}  
19       {'want': 11.402873875877027}       {'warning': 11.402873875877027}

In [36]:
d3 = pd.DataFrame(columns=columns)

d3['Count', 'Без стоп-слов'] = top_terms_stem
d3['TF', 'Без стоп-слов'] = top_terms_stem_test
d3['TF-IDF', 'Без стоп-слов'] = top_terms_notfidf_test

d3['Count', 'С стоп-словами'] = top_terms_stop
d3['TF', 'С стоп-словами'] = top_terms_tf
d3['TF-IDF', 'С стоп-словами'] = top_terms_notfidf

d3

Count                                TF                    \
     Без стоп-слов    С стоп-словами   Без стоп-слов    С стоп-словами   
0   {'the': 19591}     {'god': 1509}  {'the': 14120}     {'god': 1509}   
1    {'of': 10398}   {'space': 1003}    {'of': 7607}   {'space': 1003}   
2    {'to': 10079}   {'people': 924}    {'to': 7075}   {'people': 924}   
3    {'and': 7708}      {'don': 699}   {'and': 5459}      {'don': 699}   
4     {'is': 7025}    {'think': 685}    {'is': 4460}    {'think': 685}   
5   {'that': 6452}     {'just': 677}  {'that': 4420}     {'just': 677}   
6     {'in': 5981}     {'like': 649}    {'in': 4332}     {'like': 649}   
7     {'it': 4876}    {'jesus': 621}    {'it': 3325}    {'jesus': 621}   
8     {'be': 3511}     {'does': 613}    {'be': 2270}     {'does': 613}   
9    {'for': 3357}     {'time': 598}   {'you': 2249}     {'time': 598}   
10   {'you': 3343}     {'know': 592}   {'not': 2235}     {'know': 592}   
11   {'not': 3250}      {'say': 504}   {'for': 2114}      {'say': 504}   
12   {'thi': 2895}  {'believe': 496}   {'thi': 1912}  {'believe': 496}   
13   {'are': 2841}     {'good': 423}   {'are': 1878}     {'good': 423}   
14    {'as': 2582}      {'way': 404}    {'as': 1852}      {'way': 404}   
15  {'have': 2527}     {'nasa': 379}  {'have': 1683}     {'nasa': 379}   
16    {'on': 2372}    {'bible': 371}    {'on': 1554}    {'bible': 371}   
17   {'but': 2047}   {'things': 368}  {'with': 1428}   {'things': 368}   
18    {'do': 2044}   {'church': 362}    {'wa': 1418}   {'church': 362}   
19    {'wa': 2027}      {'new': 357}    {'or': 1331}      {'new': 357}   

                                  TF-IDF                                       
                           Без стоп-слов                       С стоп-словами  
0           {'games': 29.90811328943184}         {'games': 47.49493085644385}  
1      {'disregard': 20.705849582150545}  {'perceptions': 32.056927722331345}  
2           {'jean': 18.540420908716005}         {'thus': 29.217292991820262}  
3            {'thus': 18.40124460896598}         {'jean': 28.844933151084113}  
4    {'perceptions': 18.183976195500986}    {'disregard': 28.777536401356738}  
5         {'lasers': 17.800333147859412}       {'sphere': 28.404860402000217}  
6       {'joyfully': 16.690108932221474}       {'lasers': 26.279455417063822}  
7        {'diseases': 14.88421075732181}       {'joyfully': 25.2494597828208}  
8         {'sphere': 14.725146919122702}     {'diseases': 23.563027920793466}  
9           {'beams': 14.66841587901161}        {'iraqi': 22.530691853323308}  
10      {'tolerate': 14.058060720799915}        {'seanna': 21.01291514223008}  
11       {'charles': 13.633158812340842}         {'beams': 20.67859814888153}  
12          {'week': 13.183537910602537}     {'tolerate': 20.617870223919386}  
13  {'characterize': 13.031761008272525}        {'behind': 17.91556141148969}  
14       {'designs': 12.685464575051753}          {'gay': 17.841453750784684}  
15           {'gay': 12.593359632738883}     {'thursday': 16.869468530029685}  
16           {'sma': 12.316941351284537}      {'charles': 16.207285629300145}  
17        {'seanna': 12.231232969290474}         {'week': 16.158158902541853}  
18         {'iraqi': 11.647572890307613}     {'scenario': 15.694847716790656}  
19       {'warning': 11.402873875877027}      {'designs': 15.561100251782328}

In [37]:
d4 = pd.DataFrame(columns=columns)

d4['Count', 'Без стоп-слов'] = top_terms_stem_test
d4['TF', 'Без стоп-слов'] = top_terms_tf_test
d4['TF-IDF', 'Без стоп-слов'] = top_terms_tfidf_test

d4['Count', 'С стоп-словами'] = top_terms_stem_test
d4['TF', 'С стоп-словами'] = top_terms_tfidf_test
d4['TF-IDF', 'С стоп-словами'] = top_terms_notfidf_test

d4

Count                                  TF  \
     Без стоп-слов  С стоп-словами       Без стоп-слов   
0   {'the': 14120}  {'the': 14120}       {'god': 1023}   
1     {'of': 7607}    {'of': 7607}     {'people': 544}   
2     {'to': 7075}    {'to': 7075}        {'don': 487}   
3    {'and': 5459}   {'and': 5459}       {'like': 461}   
4     {'is': 4460}    {'is': 4460}       {'just': 444}   
5   {'that': 4420}  {'that': 4420}      {'think': 443}   
6     {'in': 4332}    {'in': 4332}       {'know': 416}   
7     {'it': 3325}    {'it': 3325}       {'does': 385}   
8     {'be': 2270}    {'be': 2270}       {'time': 376}   
9    {'you': 2249}   {'you': 2249}        {'say': 341}   
10   {'not': 2235}   {'not': 2235}    {'believe': 329}   
11   {'for': 2114}   {'for': 2114}     {'church': 323}   
12   {'thi': 1912}   {'thi': 1912}      {'space': 321}   
13   {'are': 1878}   {'are': 1878}        {'way': 321}   
14    {'as': 1852}    {'as': 1852}      {'jesus': 294}   
15  {'have': 1683}  {'have': 1683}     {'christ': 283}   
16    {'on': 1554}    {'on': 1554}        {'did': 280}   
17  {'with': 1428}  {'with': 1428}       {'good': 264}   
18    {'wa': 1418}    {'wa': 1418}        {'new': 246}   
19    {'or': 1331}    {'or': 1331}  {'christian': 244}   

                                                                  TF-IDF  \
                       С стоп-словами                      Без стоп-слов   
0          {'god': 29.90811328943184}         {'god': 29.90811328943184}   
1         {'don': 20.705849582150545}        {'don': 20.705849582150545}   
2        {'just': 18.540420908716005}       {'just': 18.540420908716005}   
3        {'think': 18.40124460896598}       {'think': 18.40124460896598}   
4      {'people': 18.183976195500986}     {'people': 18.183976195500986}   
5        {'like': 17.800333147859412}       {'like': 17.800333147859412}   
6        {'know': 16.690108932221474}       {'know': 16.690108932221474}   
7         {'does': 14.88421075732181}        {'does': 14.88421075732181}   
8       {'space': 14.725146919122702}      {'space': 14.725146919122702}   
9      {'believe': 14.66841587901161}     {'believe': 14.66841587901161}   
10       {'time': 14.058060720799915}       {'time': 14.058060720799915}   
11     {'church': 13.633158812340842}     {'church': 13.633158812340842}   
12        {'way': 13.183537910602537}        {'way': 13.183537910602537}   
13  {'christian': 13.031761008272525}  {'christian': 13.031761008272525}   
14        {'did': 12.685464575051753}        {'did': 12.685464575051753}   
15       {'good': 12.593359632738883}       {'good': 12.593359632738883}   
16        {'sin': 12.316941351284537}        {'sin': 12.316941351284537}   
17        {'say': 12.231232969290474}        {'say': 12.231232969290474}   
18      {'jesus': 11.647572890307613}      {'jesus': 11.647572890307613}   
19       {'want': 11.402873875877027}       {'want': 11.402873875877027}   

                                          
                          С стоп-словами  
0           {'games': 29.90811328943184}  
1      {'disregard': 20.705849582150545}  
2           {'jean': 18.540420908716005}  
3            {'thus': 18.40124460896598}  
4    {'perceptions': 18.183976195500986}  
5         {'lasers': 17.800333147859412}  
6       {'joyfully': 16.690108932221474}  
7        {'diseases': 14.88421075732181}  
8         {'sphere': 14.725146919122702}  
9           {'beams': 14.66841587901161}  
10      {'tolerate': 14.058060720799915}  
11       {'charles': 13.633158812340842}  
12          {'week': 13.183537910602537}  
13  {'characterize': 13.031761008272525}  
14       {'designs': 12.685464575051753}  
15           {'gay': 12.593359632738883}  
16           {'sma': 12.316941351284537}  
17        {'seanna': 12.231232969290474}  
18         {'iraqi': 11.647572890307613}  
19       {'warning': 11.402873875877027}

In [43]:
# Наивный Байесовский классификатор
def stemn(text):
    porter_stemmer = PorterStemmer()
    nltk_tokens = nltk.word_tokenize(text)
    stems = [porter_stemmer.stem(word) for word in nltk_tokens]
    return ' '.join(stems)

# Создание и обучение конвейеров 
pipelines = {
    "Стемминг без удаления стоп-слов": Pipeline([
        ('vect', CountVectorizer(analyzer=stemn)),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', MultinomialNB())
    ]),
    "С отсечением стоп-слов": Pipeline([
        ('vect', CountVectorizer(stop_words='english')),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', MultinomialNB())
    ]),
    "Стемминг c отсечением стоп-слов": Pipeline([
        ('vect', CountVectorizer(analyzer=stemn, stop_words='english')),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', MultinomialNB())
    ]),
}

# Оценка конвейеров
results = {}
for name, pipeline in pipelines.items():
    pipeline = pipeline.fit(twenty_train.data, twenty_train.target)
    prediction = pipeline.predict(twenty_test.data)

    accuracy = accuracy_score(twenty_test.target, prediction)
    precision = precision_score(twenty_test.target, prediction, average='weighted')
    recall = recall_score(twenty_test.target, prediction, average='weighted')
    f1 = f1_score(twenty_test.target, prediction, average='weighted')

    results[name] = {"Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1}

# Вывод результатов
for name, metrics in results.items():
    print(f"Метрики производительности для {name}:")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  Precision: {metrics['Precision']:.4f}")
    print(f"  Recall: {metrics['Recall']:.4f}")
    print(f"  F1 Score: {metrics['F1 Score']:.4f}")
    print()             

C:\Users\rewaz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:547: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


Метрики производительности для Стемминг без удаления стоп-слов:
  Accuracy: 0.4329
  Precision: 0.6040
  Recall: 0.4329
  F1 Score: 0.3626

Метрики производительности для С отсечением стоп-слов:
  Accuracy: 0.7291
  Precision: 0.8090
  Recall: 0.7291
  F1 Score: 0.6938

Метрики производительности для Стемминг с отсечением стоп-слов:
  Accuracy: 0.4329
  Precision: 0.6040
  Recall: 0.4329
  F1 Score: 0.3626

